In [2]:
import torch
import numpy as np
# from utils import compute_ideal_dcg, ndcg

In [3]:
y_true = torch.LongTensor([[5, 3, 2, 1, 1]]).reshape(-1, 1)
y_pred = torch.FloatTensor([3.2, 0.4, -0.1, -2.1, .5, .01]).reshape(-1, 1)

In [4]:
y_pred

tensor([[ 3.2000],
        [ 0.4000],
        [-0.1000],
        [-2.1000],
        [ 0.5000],
        [ 0.0100]])

In [6]:
%%latex
$$\lambda = \left(0.5 * (1 - S_{ij}) - \frac {1} {1 + e^{s_i - s_j}}\right) |\Delta nDCG|$$

<IPython.core.display.Latex object>

In [7]:
%%latex
$$\Delta nDCG = \frac {1} {IdealDCG} (2^i - 2^j) \left(\frac {1} {log_2(1+i)} - \frac {1} {log_2(1+j)}\right)$$

<IPython.core.display.Latex object>

In [ ]:
def compute_lambdas(y_true, y_pred, ndcg_scheme='exp2'):
    # рассчитаем нормировку, IdealDCG
    ideal_dcg = compute_ideal_dcg(y_true, ndcg_scheme=ndcg_scheme)
    N = 1 / ideal_dcg
    
    # рассчитаем порядок документов согласно оценкам релевантности
    _, rank_order = torch.sort(y_true, descending=True, axis=0)
    rank_order += 1